In [ ]:
import pandas as pd
import xgboost as xgb

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder

In [ ]:
df = pd.read_csv('tourneyGames.csv')
le_win = LabelEncoder()
le_team = LabelEncoder()
df['winner']= le_win.fit_transform(df['winner'])
df['TEAM_higher']= le_team.fit_transform(df['TEAM_higher'])
df['TEAM_lower']= le_team.fit_transform(df['TEAM_lower'])
# df = df.drop(columns=['Unnamed: 0','Year','losing_abbr','losing_name','winning_name','POSTSEASON_higher','POSTSEASON_lower','ETO_Avg_higher', 'ETO_Avg_lower','E_diff_higher','E_diff_lower'])

In [ ]:
x = df.drop('winner', axis=1)
y = df['winner']

x_train, x_test, y_train, y_test = train_test_split( x, y, test_size=0.2)

SkLearn Random Forest

In [ ]:
rf = RandomForestClassifier()
rf.fit( x_train, y_train )

In [ ]:
y_pred = rf.predict(x_test)
print(y_pred)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

XGBoost

In [ ]:
model = xgb.XGBClassifier()
model.fit(x_train,y_train)

In [ ]:
xgb_y_pred = model.predict(x_test)
print(xgb_y_pred)

In [ ]:
xgbaccuracy = accuracy_score(y_test, xgb_y_pred)
print("Accuracy:", xgbaccuracy)

Real Test:

In [ ]:
realTest = pd.read_csv('matchups25.csv')
le_test = LabelEncoder()

realTest['TEAM_higher'] = le_test.fit_transform(realTest['TEAM_higher'])
realTest['TEAM_lower'] = le_test.fit_transform(realTest['TEAM_lower'])
test = realTest.drop('winner', axis=1)

In [ ]:
# XGB Prediction
xgb_test = model.predict(test)
print(le_win.inverse_transform(xgb_test))

In [ ]:
# SK Learn Prediction
rf_test = rf.predict(test)
print(le_win.inverse_transform(rf_test))